In [ ]:
pip install rouge-score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=d484274f1905a708648eb129ed5e6c173f231a2821843ca31961693d8057e353
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge-score


In [ ]:
# This Python file uses the following encoding: utf-8

import nltk
from nltk import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import math
import codecs
import pandas as pd
import re
from rouge_score import rouge_scorer


def ROUGE2_pre(g, h):

    ac_words1 = g.split(' ')
    pro_words1 = h.split(' ')


    ac_words = list(nltk.bigrams(ac_words1))
    pro_words = list(nltk.bigrams(pro_words1))


    count = 0
    if len(ac_words) > len(pro_words):
        for x in pro_words:
            for k in ac_words:
               if x == k:
                    count= count + 1
                    break
    elif len(pro_words)>len(ac_words):
        for x in ac_words:
            for k in pro_words:
                if x == k:
                    count = count + 1
                    break

    pre_rouge_2 = count / len(pro_words)

    return pre_rouge_2



def ROUGE2_re(g, h):


    ac_words1 = g.split(' ')
    pro_words1 = h.split(' ')


    ac_words = list(nltk.bigrams(ac_words1))
    pro_words = list(nltk.bigrams(pro_words1))

    count = 0
    if len(ac_words) > len(pro_words):
        for x in pro_words:
            for k in ac_words:
                if x == k:
                    count = count + 1
                    break
    elif len(pro_words) > len(ac_words):
        for x in ac_words:
            for k in pro_words:
                if x == k:
                    count = count + 1
                    break

    # Storing the final accuracy of Rouge-1
    if(len(ac_words)==0):
      re_rouge_2 = count / 1
    else:
      re_rouge_2 = count / len(ac_words)

    return re_rouge_2




def ROUGE1_pre(g, h):

    ac_words = g.split(' ')
    pro_words = h.split(' ')


    count = 0
    if len(ac_words) > len(pro_words):
        for x in pro_words:
            for k in ac_words:
               if x == k:
                    count= count + 1
                    break
    elif len(pro_words)>len(ac_words):
        for x in ac_words:
            for k in pro_words:
                if x == k:
                    count = count + 1
                    break


    pre_rouge_1 = count / len(pro_words)

    return pre_rouge_1


def ROUGE1_re(g, h):

    ac_words = g.split(' ')
    pro_words = h.split(' ')

    count = 0
    if len(ac_words) > len(pro_words):
        for x in pro_words:
            for k in ac_words:
                if x == k:
                    count = count + 1
                    break
    elif len(pro_words) > len(ac_words):
        for x in ac_words:
            for k in pro_words:
                if x == k:
                    count = count + 1
                    break

    if(len(ac_words)==0):
      re_rouge_1 = count / 1
    else:
      re_rouge_1 = count / len(ac_words)

    return re_rouge_1







def stopwordremoval(text):

    dfsw = pd.read_csv('Stop_words.csv')
    words = dfsw.s_words.values.tolist()
    tokens = text.split(' ')

    filtered_sentence = []
    for w in tokens:
        if w not in words:
            filtered_sentence.append(w)
    new_text = ' '.join(filtered_sentence)
    return new_text


def sen_maker(text):
    sen = text.replace('۔', '%%%')
    sen = sen.replace('\r\n', ' ')
    sen = sen.split('%%%')
    return sen





def createfrequencytable(text_string) -> dict:
    dfsw = pd.read_csv('Stop_words.csv')
    stopWords = dfsw.s_words.values.tolist()
    words = text_string.split(' ')
    ps = PorterStemmer()
    freqTable = dict()
    for word in words:
        word = str(word)
        word = ps.stem(word)
        if word in stopWords:
            continue
        if word in freqTable:
            freqTable[word] += 1
        else:
            freqTable[word] = 1
    return freqTable




def scoresentences(sentences, freqTable) -> dict:
    sentenceValue = dict()
    for sentence in sentences:
        word_count_in_sentence = (len(sentence.split(' ')))
        for wordValue in freqTable:
            if wordValue in sentence.lower():
                if sentence[:10] in sentenceValue:
                    sentenceValue[sentence[:10]] += freqTable[wordValue]
                else:
                    sentenceValue[sentence[:10]] = freqTable[wordValue]
    sentenceValue[sentence[:10]] = sentenceValue[sentence[:10]] // word_count_in_sentence
    return sentenceValue



def findaverage_score(sentenceValue) -> int:
    sumValues = 0
    for entry in sentenceValue:
        sumValues += sentenceValue[entry]
    average = int(sumValues / len(sentenceValue))
    return average





def has_numbers(inputString):
    return any(char.isdigit() for char in inputString)


def noun(text):
    dfPOS = pd.read_csv('POS_Nouns.csv')
    term = dfPOS.Term.values.tolist()
    text = stopwordremoval(text)
    a = re.split(' ', text)
    j = 0
    for w in a:
        if w in term:
            return 1




def word_prob(sentence,ftable):
    sentence = stopwordremoval(sentence)
    words_sen = sentence.split(' ')
    max_term = max(ftable, key=ftable.get)
    max_freq = ftable[max_term]
    w_prob = 0
    if max_freq == 0:
        max_freq = 1
    for word in words_sen:
        if ftable.get(word):
            wp = ftable[word] / max_freq
            if wp > 0.5:
                w_prob = 1
    return w_prob






def _generate_summary(sentences, sentenceValue, threshold, ft):
    sentence_count = 0
    summary = ''
    for sentence in sentences:
        if ((sentence[:10] in sentenceValue and sentenceValue[sentence[:10]] > (threshold)) or (has_numbers(sentence)) or ((word_prob(sentence,ft)) and (noun(sentence))) ):
            summary += "۔" + sentence
            sentence_count += 1
    return summary





def create_summary(text1):
    text = text1
    ft = createfrequencytable(text)
    sentences = sen_maker(text)  # NLTK function
    total_documents = len(sentences)

    sentence_val = scoresentences(sentences, ft)

    thresh = findaverage_score(sentence_val)

    summary = _generate_summary(sentences, sentence_val, 2 * thresh, ft)
    return summary





df = pd.read_csv('Urdu_7_Books.csv')


X = df.Text
y = df.Extractive_summaries
    #df.Khulasay
z = df.Markazi_Khayal

j = 0
precount = 1
presumscore = 0
preavg = 0

resumscore = 0
reavg = 0

f_score = 0
f_sums = 0
f_avg = 0



presumscore2 = 0
preavg2 = 0

resumscore2 = 0
reavg2 = 0

f_score2 = 0
f_sums2 = 0
f_avg2 = 0

for x in X:
    XC = create_summary(str(' '+X[j]+' '))


    #======= ROUGE-1 Scores=============================


    precision = ROUGE1_pre(XC, y[j])
    recall = ROUGE1_re(XC, y[j])

    presumscore = presumscore + precision
    resumscore = resumscore + recall

    preavg = presumscore / precount
    reavg = resumscore / precount

    #==========ROUGE-2==========================================



    precision2 = ROUGE2_pre(XC, y[j])
    recall2 = ROUGE2_re(XC, y[j])
    #print(XC)

    presumscore2 = presumscore2 + precision2
    resumscore2 = resumscore2 + recall2

    preavg2 = presumscore2 / precount
    reavg2 = resumscore2 / precount

    precount = precount + 1
    j = j + 1




f_score = 2 * (preavg * reavg) / (preavg + reavg)

f_score2 = 2 * (preavg2 * reavg2) / (preavg2 + reavg2)




print("average recall is: ")
print(reavg)

print("average Precision is: ")
print(preavg)

print("f-measure average: ")
print(f_score)

print("for ROUGE 2")

print("ROUGE-2 average Precision is: ")
print(preavg2)


print("AROUGE-2 average recall is: ")
print(reavg2)

print("ROUGE-2 f-measure average: ")
print(f_score2)




average recall is: 
0.6696207223501107
average Precision is: 
0.7087207921579842
f-measure average: 
0.6886161721084617
for ROUGE 2
ROUGE-2 average Precision is: 
0.5090335884244451
AROUGE-2 average recall is: 
0.4792975590467836
ROUGE-2 f-measure average: 
0.49371823812071874
